In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [ ]:
Ly = 2.*np.pi
pth_zero = './shearflow_savez_0030/pst-n-pe-0030.npz'
data0 = np.load(pth_zero)
try:
    nsav = data0['nhst']
except KeyError:
    nsav = data0['nsav']
nx = data0['nx']; dy = data0['dx']
navg0 = np.sum(nsav[-1,:,:]*dy,axis=0)/Ly; navg0 = navg0/np.max(navg0)

In [ ]:
def sigma_calc(file_pth):
    data = np.load(file_pth)
    try:
        nsav = data['nhst']
    except KeyError:
        nsav = data['nsav']
    nx = data['nx']; dy = data['dx']
    navg = np.sum(nsav[-1,:,:]*dy,axis=0)/Ly; navg = navg/np.max(navg)
    sigmasq = np.sum((navg-navg0)**2*dy)
    return sigmasq

In [ ]:
def sheardata(sav):
    data = np.load(sav)
    # print(data.files)
    Ly = 2.*np.pi
    try:
        nsav = data['nhst']
    except KeyError:
        nsav = data['nsav']
    nx = data['nx']
    dy = data['dx']
    D = data['D']
    w = data['w']; v = data['v']
    x = np.linspace(0,Ly,nx)
    X, Y = np.meshgrid(x,x)
    navg = np.sum(nsav[-1,:,:]*dy,axis=0)/Ly
    nmax = np.max(navg)
    return nsav, navg, nmax, w, v, x , X, Y

In [ ]:
from scipy import special
f = sorted(glob.glob('./shearflow_savez_0030/shear_updated/pst-n-pe-0030-m-02-alpha-*.npz'))
r = np.linspace(-3,3,1000); distr0 = 0.5*(1+special.erf(r/np.sqrt(0.00000001*2)))
fig, ax = plt.subplots()
# ax[0].plot(r,distr0,color='black')
for i,j in enumerate(f):
    sigmasq = sigma_calc(j)
    print(np.sqrt(sigmasq))
    # gfit = (1/(np.sqrt(2*np.pi*sigmasq)))*np.exp(-0.5*r**2/sigmasq)
    distr = 0.5*(1+special.erf(r/np.sqrt(sigmasq*2)))
    ax.plot(r,distr,label=r'$\alpha = $'+str(i+1))
    # ax[1].plot(r,gfit,label=r'$\alpha = $'+str(i+1))
ax.legend(loc='best',shadow=True)
ax.set_ylim(0,1.05); ax.set_xlim(-3,3)
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
val_list = [2,4,6]; pth = './'
fig, ax = plt.subplots(ncols=len(val_list),figsize=(12,4))
m = 2
if m < 10:
    sav = sorted(glob.glob(pth+'shearflow_savez_0030\shear-m-0'+str(m)+'\*'))
else:
    sav = sorted(glob.glob(pth+'shearflow_savez_0030\shear-m-'+str(m)+'\*'))
for i in range(len(val_list)):
    nsav, navg, nmax, w, v, x, X, Y = sheardata(sav[val_list[i]])
    ax[i].pcolormesh(X,Y,nsav[-1,:,:]/np.max(nsav[-1,:,:]),vmin=0,vmax=1,cmap='gray')
    # ax[i].streamplot(X, Y, w, v,density=1,color='white',linewidth=0.2)
    ax[i].set_xlim(0,2.*np.pi); ax[i].set_ylim(0,2.*np.pi)
    ax[i].set_xticks([]); ax[i].set_yticks([])
    ax[i].set_title(r'$m$ = ' + str(m)+'; ' + r'$\eta$ = ' + str(val_list[i]))
    ax[i].set_xlabel(r'$x$',fontsize=18); ax[i].set_ylabel(r'$y$',fontsize=18)
plt.tight_layout()
plt.show()